# 1. TCP Comlink

In [ ]:
# Imports and Setup
import asyncio
import logging

from pylabrobot.io.capture import start_capture, stop_capture, capturer
from pylabrobot.liquid_handling.backends.hamilton.nimbus import TcpComLinkProxy, NIMBUSCOREDLL, COMLINKDLL, PROTOCOLSDLL
import System # type: ignore

ip_address = "192.168.100.100"
port = 2000

#start_capture() # Pylabrobot logging for comlink traffic

#Setup instrument connection
proxy = TcpComLinkProxy(ip_address, port, client_id="test_client")
# Run commands
await proxy.setup()


In [2]:
# Get modules
modules = await proxy.get_modules()
print(f"  Found {len(modules)} modules:")
# Get module information and setup NimbusCORE Control Module
for module in modules:
    print(f"    - {module.Name} (Address: {module.Address})")

    if module.Name == "NimbusCORE":
        print("\tCreating NimbusCore Instance")
        NimbusCORESystem = NIMBUSCOREDLL.GetType("Hamilton.NimbusCORE.NimbusCORESystem")
        nimbus_core = System.Activator.CreateInstance(NimbusCORESystem,
                                               proxy.get_comlink(),
                                               module.Address)

# Get number of channels
print(f'Number of channels: {nimbus_core.NimbusCORE.GetChannelConfiguration()[0]}')
# Getting Pipetting Sub-Module from NimbusCore Control
pipettor = nimbus_core.NimbusCORE.Pipettor


  Found 4 modules:
    - NimbusCORE (Address: 0x0001.0x0001.0xBF00)
	Creating NimbusCore Instance
    - NimbusCOREGlobalObjects (Address: 0x0001.0x0001.0xBF01)
    - NimbusIoBoardGlobalObjects (Address: 0x0060.0x0001.0xBF01)
    - IoBoard (Address: 0x0060.0x0001.0xBF00)
Number of channels: 4


In [3]:
# Get instrument and door lock status
door_lock_object = nimbus_core.NimbusCORE.DoorLock
print(f"   ✓ Found door lock object: {type(door_lock_object)}")

# Check door solenoid status
is_door_locked = door_lock_object.IsDoorLocked()
print(f"   ✓ Door Locked: {is_door_locked}")

if is_door_locked is False:
  # Lock the door
  door_lock_object.LockDoor()
  print("   ✓ Door locked successfully")

   ✓ Found door lock object: <class 'Hamilton.NimbusCORE.CNimbusDoorLock'>
   ✓ Door Locked: False
   ✓ Door locked successfully


In [4]:
# Preinitialize
pipettor.PreinitializeSmart()

In [5]:
# Park channels
pipettor.Park()

In [6]:
# Check door solenoid status
is_door_locked = door_lock_object.IsDoorLocked()
print(f"   ✓ Door Locked: {is_door_locked}")

if is_door_locked is True:
  # Lock the door
  door_lock_object.UnlockDoor()
  print("   ✓ Door unlocked successfully")

   ✓ Door Locked: True
   ✓ Door unlocked successfully


In [7]:
# Close out
await proxy.stop()
#stop_capture()